In [1]:
import glob
import pandas as pd
import zipfile
from tqdm import tqdm

no_date_fields = [
    "TR.GICSINDUSTRY",
    "TR.HEADQUARTERSCOUNTRY",
    "TR.SICINDUSTRY",
    "TR.COMPANYREPORTCURRENCY",
    "TR.SICINDUSTRYCODE",
    "TR.SICINDUSTRYGROUPCODE",
    "TR.GICSSUBINDUSTRYCODE",
    "ISIN_CODE",
    "TR.GICSSECTORCODE",
    "TR.GICSINDUSTRYGROUP",
    "TR.EXCHANGECOUNTRY",
    "TR.COMPANYNAME",
    "TR.GICSINDUSTRYCODE",
    "TR.INSTRUMENTTYPE",
    "TR.GICSSECTOR",
    "TR.SICINDUSTRYGROUP",
    "TR.ISINCODE",
    "TR.GICSSUBINDUSTRY",
    "TR.TICKERSYMBOL",
    "TR.CUSIPCODE",
    "TR.REGISTRATIONCOUNTRY",
    "TR.GICSINDUSTRYGROUPCODE"
]

In [2]:
full_df = pd.DataFrame()
for path in tqdm(glob.glob("assets/Eikon/*.csv")):
    df = pd.read_csv(path)

    df[no_date_fields] = df[no_date_fields].ffill()

    for field in [c for c in df.columns if c.endswith(".DATE")]:
        df[field] = pd.to_datetime(df[field], format=r"%Y-%m-%dT%H:%M:%SZ")

    full_df = pd.concat([full_df, df], axis=0)

100%|██████████| 12658/12658 [1:07:50<00:00,  3.11it/s]


In [15]:
str_cols = [
    "TR.SEDOL",
    "ISIN_CODE",
    "TR.TICKERSYMBOL",
    "TR.CUSIPCODE",
]

for col in str_cols:
    full_df[col] = full_df[col].astype(str)

full_df.to_parquet("data.parquet")

In [11]:
full_df["TR.SEDOL"].unique()

array([6803087.0, nan, 6000156.0, ..., 'BYX23B2', 'BDC5SR6', 'BYZ2H41'],
      dtype=object)

In [16]:
pd.read_parquet("data.parquet")

,Instrument,TR.GICSINDUSTRY,TR.POLICYSUSTAINABLEPACKAGING,TR.CO2EMISSIONTOTAL,TR.PMREDUCTION,TR.F.COMEQPARENTSHHOLD,TR.HEADQUARTERSCOUNTRY,TR.ANALYTICRESOURCEREDPOLICY,TR.UPSTREAMSCOPE3TRANSPORTATIONANDDISTRIBUTION,TR.F.COGSUNCLASSIF,...,TR.VOCEMISSIONS.DATE,TR.COGSACTVALUE.DATE,TR.UPSTREAMSCOPE3LEASEDASSETS.DATE,TR.NOXEMISSIONS.DATE,TR.ANALYTICRENEWENERGYUSE.DATE,TR.F.DEFTAXINVSTTAXCREDITSLT.DATE,TR.VOCEMISSIONSREDUCTION.DATE,TR.DOWNSTREAMSCOPE3TRANSPORTATIONANDDISTRIBUTION.DATE,TR.ANALYTICWATERUSE.DATE,TR.SOXEMISSIONS.DATE
0,000004.SZ,Software,NaN,NaN,NaN,353.301463,China,NaN,NaN,104.253262,...,NaT,NaT,NaT,NaT,NaT,2022-12-31,NaT,NaT,NaT,NaT
1,000004.SZ,Software,NaN,NaN,NaN,948.958917,China,NaN,NaN,125.936719,...,NaT,NaT,NaT,NaT,NaT,2021-12-31,NaT,NaT,NaT,NaT
2,000004.SZ,Software,NaN,NaN,NaN,1458.075267,China,NaN,NaN,69.277024,...,NaT,NaT,NaT,NaT,NaT,2020-12-31,NaT,NaT,NaT,NaT
3,000004.SZ,Software,NaN,NaN,NaN,1391.202431,China,NaN,NaN,29.196174,...,NaT,NaT,NaT,NaT,NaT,2019-12-31,NaT,NaT,NaT,NaT
4,000004.SZ,Software,NaN,NaN,NaN,110.035776,China,NaN,NaN,66.478046,...,NaT,NaT,NaT,NaT,NaT,2018-12-31,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,ZZLL.PK,None,NaN,NaN,NaN,NaN,Hong Kong,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
22,ZZLL.PK,None,NaN,NaN,NaN,NaN,Hong Kong,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
23,ZZLL.PK,None,NaN,NaN,NaN,NaN,Hong Kong,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
24,ZZLL.PK,None,NaN,NaN,NaN,NaN,Hong Kong,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [ ]:
import os

df = pd.read_csv("assets\metadata\misc\REF_exchangecountry.csv")
us_firms = df[df["Country of Exchange"]=="United States of America"]["Instrument"].tolist()
path = os.path.join("assets", "Eikon")

with zipfile.ZipFile("US_firms1.zip", "w") as f:
    for firm in us_firms:
        file_name = os.path.join(path, f"{firm}.csv")
        f.write(file_name, compress_type=zipfile.ZIP_DEFLATED, arcname=f"{firm}.csv")